In [11]:
import numpy as np
import cmath

In [36]:
def QAM16(symbol):
	return -3+2*(symbol>>2)+1j*(3-2*(symbol&0b11))

def map_bytes(bytes_list):
	mapped_symbols = []
	for b in bytes_list:
		mapped_symbols += [QAM16(b>>4),QAM16(b&0b1111)]
	return mapped_symbols

def OFDM(T,N,X,t):
	return sum([X[k]*np.exp(2j*np.pi*k*t/T) for k in range(N)])


In [7]:
data = np.fromfile('test.png', dtype = "uint8")
mapped = map_bytes(data[0:0+4])
print(mapped)

In [9]:
print(type(mapped[0]))
print(mapped)

<class 'numpy.complex128'>
[(1+3j), (1+1j), (-1+1j), (-3+3j), (-1+3j), (3-1j), (-1+3j), (-1-3j)]


In [30]:
def demod_QAM16(z:complex):
    #* returns 4 bits
    symbol = int((z.real+3)*2)
    symbol |= int((3-z.imag)/2)
    return symbol

def demap_bytes(z_list):
    #* z_list should be a list of 8 complex number
    #* returns 4 bytes
    bytes_list = []
    for i in range(0, len(z_list), 2):
        B = demod_QAM16(z_list[i])<<4
        B |= demod_QAM16(z_list[i+1])
        bytes_list.append(B)
    return bytes_list

In [63]:
NB_SOUS_PORTEUSES = 8
F_C = 7e3
F_E = int(50*F_C)
T_E = 1/F_E
R = 1000
T = 1/R

In [77]:
def demod_OFDM(mod, k):
    return 1/int(F_E*T) * sum([mod[t]*np.exp(-2j*np.pi*k*t*T_E/T) for t in range(int(F_E*T))])

In [93]:
def roundComplexList(z_list):
    for i in range(len(z_list)):
        z_list[i] = round(z_list[i].real,0) + round(z_list[i].imag,0)*1j

In [94]:
raw_data = np.fromfile("flag.iq", dtype='complex64')
png_data = []
for i in range(0, len(raw_data), int(F_E*T)):
    modulated = raw_data[i:i+int(F_E*T)]
    z_list = [demod_OFDM(modulated, k) for k in range(8)]
    roundComplexList(z_list)
    bytes_list = demap_bytes(z_list)
    png_data += bytes_list

print(png_data)
f = open("flag.png", 'wb')
f.write(bytes(png_data))
f.close()

[137, 80, 78, 71, 13, 10, 26, 10, 0, 0, 0, 13, 73, 72, 68, 82, 0, 0, 2, 188, 0, 0, 0, 200, 8, 2, 0, 0, 0, 133, 244, 165, 234, 0, 0, 37, 230, 73, 68, 65, 84, 120, 156, 237, 221, 103, 92, 20, 199, 195, 7, 240, 5, 185, 3, 143, 46, 82, 4, 11, 42, 177, 98, 48, 42, 246, 174, 216, 43, 26, 99, 140, 221, 88, 242, 183, 119, 77, 20, 52, 118, 36, 118, 141, 138, 216, 177, 215, 40, 216, 99, 165, 40, 86, 20, 176, 0, 54, 4, 169, 210, 142, 114, 192, 241, 188, 88, 159, 97, 115, 141, 225, 110, 15, 142, 228, 247, 253, 248, 98, 14, 246, 246, 134, 157, 245, 238, 119, 179, 179, 51, 122, 143, 31, 23, 49, 0, 0, 0, 0, 37, 209, 47, 239, 10, 0, 0, 0, 64, 197, 128, 208, 0, 0, 0, 0, 84, 16, 26, 0, 0, 0, 128, 10, 66, 3, 0, 0, 0, 80, 65, 104, 0, 0, 0, 0, 42, 8, 13, 0, 0, 0, 64, 5, 161, 1, 0, 0, 0, 168, 32, 52, 0, 0, 0, 0, 21, 132, 6, 0, 0, 0, 160, 130, 208, 0, 0, 0, 0, 84, 16, 26, 0, 0, 0, 128, 10, 66, 3, 0, 0, 0, 80, 65, 104, 0, 0, 0, 0, 42, 8, 13, 0, 0, 0, 64, 5, 161, 1, 0, 0, 0, 168, 32, 52, 0, 0, 0, 0, 21, 132, 6